In [1]:
# import the required libraries 
import pymongo
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import seaborn as sns
import tkinter as tk
from tkinter import ttk
import time

In [2]:
# Global variables
inspections_df = 0
inventory_df = 0
violations_df = 0
df = 0
score = 0
unstack = 0
stats = 0
prep_df = 0
establishment_table = 0
correlation_table = 0
message = ''
label_file_frame5 = 0

In [3]:
# creating root for tkinter GUI, title and fixed size.


root = tk.Tk()
root.title('Summative Assessment')
root.geometry("800x700")
root.pack_propagate(False)
root.resizable(0,0)

# Frame for Treeview 
frame1 = tk.LabelFrame(root, text="Client Data")
frame1.place(height=325,width=800)

# Frame for table for inspection score - mean, median and mode
file_frame = tk.LabelFrame(root,text="Inspection Score Visualization")
file_frame.place(height=180,width=355,rely=0.48,relx=0.04)

# Frame for establishment graphic visualization 
file_frame2 = tk.LabelFrame(root,text="Establishment Violations Visualization")
file_frame2.place(height=180,width=355,rely=0.48,relx=0.52)
# label for establishment frame
label_file_frame2 = tk.Label(file_frame2,text='Establishments that have commited each type of violation')
label_file_frame2.pack()

# Frame for loading and saving dataset
file_frame3 = tk.LabelFrame(root,text='File Manager')
file_frame3.place(height=70,width=355,rely=0.877,relx=0.52)

# Frame for Zip Code and Number of violations correlation visualization
file_frame4 = tk.LabelFrame(root,text='Correlation Visualization')
file_frame4.place(height=160,width=355,rely=0.74,relx=0.04)
# Label for correlation visualization frame
label_file_frame4 = tk.Label(file_frame4,text='Please input Vendor Name:')
label_file_frame4.pack()

# Frame message output
file_frame5 = tk.LabelFrame(root)
file_frame5.place(height=80,width=355,rely=0.75,relx=0.52)
# Label for message output frame
label_file_frame5 = ttk.Label(file_frame5,text='No File Loaded')
label_file_frame5.place(rely=0,relx=0)

In [4]:
# button created to load and clean initial dataset's
load_clean_button = tk.Button(file_frame3,text="Load and clean dataset's",command=lambda: load_and_clean_data())
load_clean_button.place(rely=0.02,relx=0.08)

# button created to show treeview table for yearly mean, median and mode values for each PE DESCRIPTION
show_button = tk.Button(file_frame,text="Show Table",command=lambda: Load_csv_data())
show_button.place(rely=0.5,relx=0.4)

# button created to save cleaned/prepared dataset
save_button = tk.Button(file_frame3,text="Save prepared dataset",command=lambda: save_dataset())
save_button.place(rely=0.02,relx=0.55)

# button created to load a saved prepared dataset
load_prep_button = tk.Button(file_frame3,text="Load prepared dataset",command=lambda: load_prepared_dataset())
load_prep_button.place(rely=0.5,relx=0.33)

# function created to output message to user including a box and label
def message_box():
    global message
    global label_file_frame5
    label_file_frame5 = ttk.Label(file_frame5,text=message)
    label_file_frame5.place(rely=0,relx=0)
   
# function created to clear message box output when new message is displayed
def message_delete():
    global label_file_frame5
    label_file_frame5.destroy()

In [5]:
# Treeview Widget created for inspection score statistical values 
tv1 = ttk.Treeview(frame1)
tv1.place(relheight=1,relwidth=1)

# scrollbar created for table - enables user to view and interact with entire dataset
treescrolly = tk.Scrollbar(frame1,orient="vertical",command=tv1.yview)
treescrollx = tk.Scrollbar(frame1,orient="horizontal",command=tv1.xview)
tv1.configure(xscrollcommand=treescrollx.set, yscrollcommand=treescrolly.set)
treescrollx.pack(side="bottom",fill="x")
treescrolly.pack(side="right",fill="y")

In [6]:
# function created to convert .csv to .json
def convert_to_json(file):
    # .read_csv converts .csv into pandas dataframe
    csv_data = pd.read_csv(file) 
    # convert to dictionary and values to lists
    json_data = csv_data.to_dict('records') 
                                           
    return json_data 

In [7]:
def load_and_clean_data():
    global inspections_df 
    global inventory_df 
    global violations_df 
    global loaded
    global df
    global prep_df
    global establishment_table
    global message
    global MongoDB
    global group_zip
    
    startTime = time.time()

    # connect to mongodb client and database
    connection = pymongo.MongoClient("mongodb://localhost:27017/") 
    # create a new database 
    MongoDB = connection["data"] 

    # delete duplicate/multiple collections
    ins_collection = MongoDB["inspections"].delete_many({})
    inv_collection = MongoDB["inventory"].delete_many({})
    vio_collection = MongoDB["violations"].delete_many({})
    
    # new collections for each file to be loaded into the database
    ins_collection = MongoDB["inspections"]
    inv_collection = MongoDB["inventory"]
    vio_collection = MongoDB["violations"]

    # insert json data into their respective collections
    ins_collection.insert_many(convert_to_json('Inspections.csv'))
    inv_collection.insert_many(convert_to_json('Inventroy.csv'))
    vio_collection.insert_many(convert_to_json('Violations.csv'))

    # individually convert all 3 collections into their own dataframes
    # '_id' as false to remove the mongodb id references from each item
    inspections_df = pd.DataFrame(list(ins_collection.find({}, {'_id': False})))
    inventory_df = pd.DataFrame(list(inv_collection.find({}, {'_id': False})))
    violations_df = pd.DataFrame(list(vio_collection.find({}, {'_id': False})))
    

    # remove duplicate 
    inspections_df.drop_duplicates(inplace=True) 
    inventory_df.drop_duplicates(inplace=True) 
    violations_df.drop_duplicates(inplace=True) 
    # Keep the required columns (columns that do not exist in inspections)
    inventory_df = inventory_df[['RECORD ID','Zip Codes']]
    # merge dataframe 
    df = pd.merge(inventory_df,inspections_df,on=['RECORD ID'],how='right')

    # fill in missing data for zipcodes from Inventory into Inspections
    df["Zip Codes"] = df["Zip Codes_x"].combine_first(df["Zip Codes_y"])
    df = df.drop(["Zip Codes_x", "Zip Codes_y"], axis=1)

    # Filter dataset to ACTIVE only
    df = df.loc[df['PROGRAM STATUS'] == 'ACTIVE']

    # Remove missing values
    df = df[df['SCORE'].notna()]
    df = df[df['Zip Codes'].notna()]
        
    #Filter Dataframes
 
    df = df[['FACILITY NAME','ACTIVITY DATE','SCORE','PROGRAM STATUS','PE DESCRIPTION','Zip Codes','SERIAL NUMBER']]

    # extract information within brackets using regular expressions to create a new column
    df['PE CODE'] = df['PE DESCRIPTION'].str.extract(r"\((.*?)\)", expand=False)
    df['PE DESCRIPTION'] = df['PE DESCRIPTION'].str.replace(r"\((.*?)\)","")

    # fix description name for LIC HTH CARE FOOD FAC HIGH RISK due to double spacing
    df["PE DESCRIPTION"]= df["PE DESCRIPTION"].replace('LIC HTH CARE FOOD FAC  HIGH RISK', 
                                                       'LIC HTH CARE FOOD FAC HIGH RISK',inplace=False) 
    # convert activity date to datetime and create new column to sort by year
    df['ACTIVITY DATE'] = pd.to_datetime(df['ACTIVITY DATE'])
    ACT_YEAR = df['ACTIVITY DATE'].map(lambda x: x.year)
    df['ACT_YEAR'] = ACT_YEAR
    df['Zip Codes'] = df['Zip Codes'].astype(int)

    # Create prepared dataframe
    prep_df = pd.merge(violations_df,df,on=['SERIAL NUMBER'],how='right')
    prep_df = prep_df[prep_df['VIOLATION CODE'].notna()]
    group_zip = prep_df.copy()
    group_zip['Zip Codes'] = group_zip['Zip Codes'].astype(str).str[:-2].astype(np.int64)
    
    message = "Dataset's loaded and cleaned, ready to save."
    message_box()   
    
    executionTime = (time.time() - startTime)
    
    print('Execution time in seconds: ' + str(executionTime))

In [8]:
def save_dataset():
    
    global message
    global label_file_frame5
    
    connection = pymongo.MongoClient("mongodb://localhost:27017/") #connect to local mongodb database
    #connection.list_database_names()
    try:
        MongoDB = connection["data"]
        coll_names = MongoDB.list_collection_names()

        if ('prepared data' not in coll_names) and (isinstance(prep_df,pd.DataFrame)):
            save()
            label_file_frame5.destroy()
            message = 'Saved cleaned dataset'
            message_box()

        elif ('prepared data' in coll_names) and (isinstance(prep_df,pd.DataFrame)):
            save()
            label_file_frame5.destroy()
            message = 'Existing prepared dataset overwrited'
            message_box()

        else:
            pass
    except:
        message_delete()
        message = 'Prepared data already exists'
        message_box()
    
def save():
    
    saved_collection = MongoDB["prepared data"].delete_many({})
    saved_collection = MongoDB["prepared data"]
    df_save = prep_df.to_json(orient="records")
    parsed = json.loads(df_save)
    saved_collection.insert_many(parsed)

In [9]:
def load_prepared_dataset():
    
    global prep_df
    global message
    global label_file_frame5
    global group_zip
    
    connection = pymongo.MongoClient("mongodb://localhost:27017/") #connect to local mongodb database
    MongoDB = connection["data"]
    
    coll_names = MongoDB.list_collection_names()
    
    if 'prepared data' not in coll_names:
        message_delete()
        message = 'Prepared data does not exist'
        message_box()
        
    elif 'prepared data' in coll_names:
        
        prepared_dataset = MongoDB["prepared data"]
        prep_df = pd.DataFrame(list(prepared_dataset.find({}, {'_id': False})))
        group_zip = prep_df.copy()
        group_zip['Zip Codes'] = group_zip['Zip Codes'].astype(str).str[:-2].astype(np.int64)
        
        message_delete()
        message = 'Prepared data loaded'
        message_box()

In [10]:
options = [
           "Vendor's seating - Mean inspection score per year",
           "Vendor's seating - Mode inspection score per year",
           "Vendor's seating - Median inspection score per year",
            'Zip Code - Mean inspection score per year',
            'Zip Code - Mode inspection score per year',
            'Zip Code - Median inspection score per year',]

clicked = tk.StringVar()
clicked.set(options[0])

drop = tk.OptionMenu(file_frame, clicked, *options)
drop.place(rely=0.2,relx=0.01)

def Load_csv_data():

    global score
    global stats
    global message
    global label_file_frame5
   
    options_clicked = clicked.get()
    
    try:
        if options_clicked == "Vendor's seating - Mean inspection score per year":
            stats = prep_df.groupby(['ACT_YEAR','PE DESCRIPTION'])[['SCORE']].mean()
            pe_description_data_manipulate()

        elif options_clicked == "Vendor's seating - Mode inspection score per year":
            stats = prep_df.groupby(['ACT_YEAR','PE DESCRIPTION'])[['SCORE']].agg(lambda x:x.value_counts().index[0])
            pe_description_data_manipulate()

        elif options_clicked == "Vendor's seating - Median inspection score per year":
            stats = prep_df.groupby(['ACT_YEAR','PE DESCRIPTION'])[['SCORE']].median()
            pe_description_data_manipulate()

        elif options_clicked == 'Zip Code - Mean inspection score per year':
            stats = prep_df.groupby(['ACT_YEAR','Zip Codes'])[['SCORE']].mean()
            zipcode_data_manipulate()

        elif options_clicked == 'Zip Code - Mode inspection score per year':
            stats = prep_df.groupby(['ACT_YEAR','Zip Codes'])[['SCORE']].agg(lambda x:x.value_counts().index[0])
            zipcode_data_manipulate()

        elif options_clicked == 'Zip Code - Median inspection score per year':
            stats = prep_df.groupby(['ACT_YEAR','Zip Codes'])[['SCORE']].median()
            zipcode_data_manipulate()

        clear_data()
        tv1["column"] = list(score.columns)
        tv1["show"] = "headings"

        for column in tv1["columns"]:
            tv1.heading(column,text=column)

        df_rows = score.to_numpy().tolist()
        for row in df_rows:
            tv1.insert("","end",values=row)
        return None
    
    except:
        message_delete()
        message = 'Please load new dataset or a prepared dataset'
        message_box()

def zipcode_data_manipulate():
    
    global score
    global unstack
    global stats
    global message
    global label_file_frame5
    
    try:
        unstack = stats.unstack(0).reset_index()
        score = unstack['SCORE'].round(2)
        score['Zip Codes'] = unstack['Zip Codes']
        first_col =score.pop('Zip Codes')
        score.insert(0,'Zip Codes',first_col)
        score.columns.name = None
        score.reset_index(drop=True, inplace=True)
        
    except: 
        message_delete()
        message = 'Please load new dataset or a prepared dataset'
        message_box()
    
def pe_description_data_manipulate():
    
    global score
    global unstack
    global stats
    global message
    global label_file_frame5
    
    try:
        unstack = stats.unstack(0).reset_index()
        score = unstack['SCORE'].round(2)
        score['PE DESCRIPTION'] = unstack['PE DESCRIPTION']
        first_col = score.pop('PE DESCRIPTION')
        score.insert(0,'PE DESCRIPTION',first_col)
        score.columns.name = None
        score.reset_index(drop=True, inplace=True)
    except:
        message_delete()
        message = 'Please load new dataset or a prepared dataset'
        message_box()
    
def clear_data():
    tv1.delete(*tv1.get_children())

In [11]:
clicked2 = tk.StringVar()
clicked2.set(None)

MODES = [
    ("10000+","10000+"),
    ("1000-10000","1000-10000"),
    ("100-1000","100-1000"),
    ("2-100","2-100"),
    ("1","1")
]

for text, mode in MODES:
    tk.Radiobutton(file_frame2,text=text, variable=clicked2,value=mode).pack(anchor=tk.N)
    
button3 = tk.Button(file_frame2,text="Load Graph",command=lambda: graphvis())
button3.pack()

def graphvis():
    #global variables for message output
    global message
    global label_file_frame5
    # try-except implemented for error handling - when dataset is not loaded by user 
    try:
        # dataframe manipulation for matplotlib visualisation, multi-index level reset and values sorted.
        establishment_table = prep_df.groupby(['VIOLATION CODE'])[['FACILITY NAME']].nunique()
        establishment_table = establishment_table.reset_index()
        establishment_table = establishment_table.sort_values('FACILITY NAME',ascending = False)
        
        # variable assigned for grabbing the string variable for radio button selected.
        mode_clicked = clicked2.get()
        rnge = 0
        
        # creating a new tkinter window for matplotlib graph, set the title and window size. 
        graph = tk.Tk()
        graph.title('Number of establishments that have commited each type of violation')
        graph.geometry("1200x600")

        # group the data into different ranges to make visualisation’s feasible
        highest = establishment_table[(establishment_table['FACILITY NAME'] >= 10000)]
        high = establishment_table[(establishment_table['FACILITY NAME'] <= 10000) & 
                                   (establishment_table['FACILITY NAME'] >= 1000) ]
        mid = establishment_table[(establishment_table['FACILITY NAME'] <= 1000) & 
                                  (establishment_table['FACILITY NAME'] >= 100) ]
        low = establishment_table[(establishment_table['FACILITY NAME'] <= 100) & 
                                  (establishment_table['FACILITY NAME'] >= 2) ]
        lowest = establishment_table[(establishment_table['FACILITY NAME'] < 2)]
        
        # if statement to select different ranges according to a set of conditions
        if mode_clicked == "10000+":
            rnge = highest
        elif mode_clicked == "1000-10000":
            rnge = high
        elif mode_clicked == "100-1000":
            rnge = mid 
        elif mode_clicked == "2-100":
            rnge = low
        elif mode_clicked == "1":
            rnge = lowest
        
        # Create matplotlib figure and set the size.
        fig = plt.Figure(figsize=(50,10),dpi=100)
        # Bar chart style selected, x and y axis are plotted, additional styling and renaming applied 
        fig.add_subplot(111).bar(rnge['VIOLATION CODE'],rnge['FACILITY NAME'],color='blue')
        fig.autofmt_xdate(rotation=90)
        fig.text(0.5, 0.04, 'Violation Codes', ha='center')
        fig.text(0.04, 0.5, 'Number of Establishments', va='center', rotation='vertical')
        # Imported canvas from enabling matplotlib and tkinter integration 
        chart = FigureCanvasTkAgg(fig,graph)
        chart.get_tk_widget().pack()
        
    except:
        # Exception handling when no dataset is loaded
        message_delete()
        message = 'Please load new dataset or a prepared dataset'
        message_box()

In [12]:
clicked3 = tk.StringVar()
clicked3.set('Grouped Locations')

selection = [
    ("Grouped Locations","Grouped Locations"),
    ("Individual Locations",'Individual Locations'),
]

for text, mode in selection:
    tk.Radiobutton(file_frame4,text=text, variable=clicked3,value=mode).pack(anchor=tk.N)

# create user input box for facility/owner ID information
e = tk.Entry(file_frame4,width=30,)
e.insert(0,'subway')
e.pack()

# create a button to load the seaborn visualisation
button4 = tk.Button(file_frame4,text="Load Graph",command=lambda: correlation_plot())
button4.pack()

def correlation_plot():
    
    global message
    global label_file_frame5
    global prep_df
    global facility_input
    
    # variables for selected column names and vendor/facility names 
    locations_grouping = 0
    facility_input = 0
    
    # .get() to grab radio button string value and assign as variable
    mode_clicked = clicked3.get()
    # .get() to grab user input string and assign as variable
    facility_input = e.get()
    
    # if statement for radio button choices
    if mode_clicked == "Grouped Locations":
        locations_grouping = group_zip
    elif mode_clicked == "Individual Locations":
        locations_grouping = prep_df

    try:
        # filtering and shaping data frame for seaborn visualisation, cosisting of groupby and reset_index()
        correlation_table = locations_grouping.loc[locations_grouping['FACILITY NAME'] == facility_input.upper()]
        correlation_table = correlation_table.groupby(['FACILITY NAME','Zip Codes'])['VIOLATION CODE'].count()
        correlation_table = correlation_table.reset_index()

        # create a tkinter window for the seaborn bar chart
        graph = tk.Tk()
        graph.title("Seaborn bar chart graphical visualisation")
        graph.geometry("1200x600")
        sns.set(style="whitegrid")

        # Set up the seaborn figure 
        f, ax = plt.subplots(figsize=(50, 10))
        # Draw the barplot and label title and axis
        g = sns.barplot(x="Zip Codes", y="VIOLATION CODE", data=correlation_table)
        plt.xticks(rotation=90)
        plt.ylabel("Number of Violations")
        plt.xlabel(facility_input + ' Zip Codes')
        plt.title("Number of violations that have been commited by each " + facility_input + " location")
        # A tk.DrawingArea
        chart = FigureCanvasTkAgg(f, graph)  
        chart.get_tk_widget().pack()
    except:
        # Exception handling when no dataset is loaded
        message_delete()
        message = 'Please load new dataset or an existing prepared dataset'
        message_box()

In [ ]:
root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/eric/opt/anaconda3/lib/python3.7/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-4-61fe1aba1f97>", line 2, in <lambda>
    load_clean_button = tk.Button(file_frame3,text="Load and clean dataset's",command=lambda: load_and_clean_data())
  File "<ipython-input-7-1c143425e019>", line 31, in load_and_clean_data
    ins_collection.insert_many(convert_to_json('Inspections.csv'))
  File "<ipython-input-6-ed369cc3b99f>", line 4, in convert_to_json
    csv_data = pd.read_csv(file)
  File "/Users/eric/opt/anaconda3/lib/python3.7/site-packages/pandas/io/parsers.py", line 676, in parser_f
    return _read(filepath_or_buffer, kwds)
  File "/Users/eric/opt/anaconda3/lib/python3.7/site-packages/pandas/io/parsers.py", line 448, in _read
    parser = TextFileReader(fp_or_buf, **kwds)
  File "/Users/eric/opt/anaconda3/lib/python3.7/site-packages/pandas/io/parsers.py", line 880,